In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [14]:
%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer, kaggle_dataset_helper
from ml.models.base.HyperOpt.brisk_bagging import BriskBagging

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common
import ray
import time
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
rayer.get_global_cluster(num_cpus=10)

In [4]:
!ray status #--address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

Traceback (most recent call last):
  File "/home/vagrant/miniconda3/envs/automl/bin/ray", line 8, in <module>
    sys.exit(main())
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/ray/scripts/scripts.py", line 2596, in main
    return cli()
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/click/core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/click/core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/click/core.py", line 1395, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/click/core.py", line 754, in invoke
    return 

In [5]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)

In [15]:
ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
ds_train = common.label_encode(ds_train)
ds_test = common.label_encode(ds_test)

ds_train = ds_train.fillna(-1)
ds_test = ds_test.fillna(-1)


df_X = ds_train.loc[:, ds_train.columns != 'target']
df_y = ds_train['target']

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)



In [16]:
X_train_id = ray.put(X_train_scalar)
y_train_id = ray.put(y_train)
X_test_id = ray.put(X_train_scalar)
y_test_id = ray.put(y_test)

In [17]:

@ray.remote(num_returns=1)
def worker(base_model, X_train, X_test, y_train, y_test):     
    base_model.fetch_model(X_train, X_test, y_train, y_test)
    return base_model


In [20]:
brisk_bag_1 = BriskBagging('brisk_xgb1')
brisk_bag_1.n_estimators = 100


brisk_bag_2 = BriskBagging('brisk_xgb2')
brisk_bag_2.n_estimators = 100


brisk_bag_3 = BriskBagging('brisk_xgb3')
brisk_bag_3.n_estimators = 100


brisk_bag_4 = BriskBagging('brisk_xgb4')
brisk_bag_4.n_estimators = 100


brisk_bag_5 = BriskBagging('brisk_xgb5')
brisk_bag_5.n_estimators = 100

list_models = [brisk_bag_1, brisk_bag_2]

In [21]:
model_results = ray.get([worker.remote(base_model, X_train_id, X_test_id, y_train_id, y_test_id) for base_model in list_models])

(worker pid=8439) 2022-12-01T10:17:58UTC : INFO : brisk_bagging : __discover_model__ : 96 : Message : brisk_xgb1: Starting training for trials:300, n_estimators  100
(worker pid=8439) 2022-12-01T10:17:58UTC : INFO : tpe : suggest : 864 : Message : build_posterior_wrapper took 0.000882 seconds
(worker pid=8439) 2022-12-01T10:17:58UTC : INFO : tpe : suggest : 900 : Message : TPE using 0 trials


  0%|          | 0/100 [36:14<?, ?trial/s, best loss=?]


(worker pid=8032) 2022-12-01T10:21:49UTC : ERROR : fmin : serial_evaluate : 180 : Message : job exception: Found input variables with inconsistent numbers of samples: [30706, 62341]
(worker pid=8440) 2022-12-01T10:40:23UTC : ERROR : fmin : serial_evaluate : 180 : Message : job exception: Found input variables with inconsistent numbers of samples: [30706, 62341]


  0%|          | 0/100 [54:48<?, ?trial/s, best loss=?]


(worker pid=8441) 2022-12-01T10:55:37UTC : ERROR : fmin : serial_evaluate : 180 : Message : job exception: Found input variables with inconsistent numbers of samples: [30706, 62341]


  0%|          | 0/100 [40:13<?, ?trial/s, best loss=?]


(worker pid=8439) 2022-12-01T10:58:12UTC : ERROR : fmin : serial_evaluate : 180 : Message : job exception: Found input variables with inconsistent numbers of samples: [30706, 62341]


RayTaskError(ValueError): [36mray::worker()[39m (pid=8439, ip=192.168.0.249)
  File "/tmp/ipykernel_16988/946824343.py", line 3, in worker
  File "/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai/notebooks/hyperOpt/../../ml/models/base/HyperOpt/brisk_bagging.py", line 156, in fetch_model
  File "/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai/notebooks/hyperOpt/../../ml/models/base/HyperOpt/brisk_bagging.py", line 114, in __discover_model__
  File "/tmp/ray/session_2022-12-01_08-38-43_279183_1405/runtime_resources/pip/6315c0fe214a8b2cd0da6f759f65803033c84e03/virtualenv/lib/python3.8/site-packages/hyperopt/fmin.py", line 586, in fmin
    rval.exhaust()
  File "/tmp/ray/session_2022-12-01_08-38-43_279183_1405/runtime_resources/pip/6315c0fe214a8b2cd0da6f759f65803033c84e03/virtualenv/lib/python3.8/site-packages/hyperopt/fmin.py", line 364, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/tmp/ray/session_2022-12-01_08-38-43_279183_1405/runtime_resourc

In [13]:
model_results